# CAI Prompt

In [24]:
import os
import openai
from dotenv import load_dotenv
import concurrent.futures
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

number_prompts = 5
# "Generate a list of {tasks_description}."
tasks_description = "tasks for a customer service assistant involving reading and writing"

# "Here is a {task_description}."
task_description = "task for a customer service assistant"


In [27]:
def get_concepts_prompt(n: int):
    return f"Generate a list of {n} tasks for a customer service assistant involving reading and writing. Write each one on a separate, numbered line."


## Generate tasks/concepts

In [25]:


completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
          "role": "user",
          "content": f"Generate a list of {number_prompts} {tasks_description}. Write each one on a separate, numbered line."}
    ],
    max_tokens=1024,
    temperature=0.5,
    stream=False
)

tasks_string = completion.choices[0].message.content
print(tasks_string)


1. Responding to customer inquiries via email or chat, which involves reading the customer's message and crafting a clear and helpful response.
2. Documenting customer interactions and feedback in a CRM system, which involves writing detailed notes about the customer's issue or request and the actions taken to resolve it.
3. Proofreading and editing customer-facing materials such as FAQs, product descriptions, or help center articles to ensure they are clear, accurate, and free of errors.
4. Creating or updating templates for common customer inquiries or responses, which involves writing clear and concise language that can be easily customized for each customer.
5. Collaborating with other teams, such as product or engineering, to ensure that customer feedback is captured and shared effectively, which involves reading and summarizing customer feedback and writing clear and actionable reports or recommendations.


In [26]:
def getItems(str):
    items = []
    for line in str.splitlines():
        items.append(line[line.find(".") + 2:])
    return items


tasks = getItems(tasks_string)
print(len(tasks))


5


## Generate naive responses

> **Terminology** - here we're prompting an LLM to generate prompts, which is confusing. We'll use the word "prompt" to refer to the initial input, and "response" to refer to the output

In [5]:
def get_llm_response(task, i):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": f"Here is a {task_description}:\n\n{task}\n\nWrite a prompt for a large language model (LLM), asking it to carry out this task."
            }
        ],
        max_tokens=1024,
        temperature=0.5,
        stream=False
    )
    str = response['choices'][0]['message']['content']
    print(f"Response {i} received")
    return str


with concurrent.futures.ThreadPoolExecutor() as executor:
    naive_responses = list(executor.map(lambda x: get_llm_response(
        x[0], x[1]), [(task, i) for i, task in enumerate(tasks)]))

tasksResponsesString = "\n\n---\n\n".join(
    [f"TASK:\n{task}\n\nNAIVE PROMPT:\n{response}" for task, response in zip(tasks, naive_responses)])
print(tasksResponsesString)


Response 3 received
Response 4 received
Response 2 received
Response 1 received
Response 0 received
TASK:
Reading and responding to customer inquiries via email

NAIVE PROMPT:
Please use the following prompt for a large language model:

Task: Reading and responding to customer inquiries via email

Description: As a customer service assistant, your task is to read and respond to customer inquiries via email. Your goal is to provide timely and helpful responses to customers, addressing their concerns and questions in a professional and courteous manner. You will need to be able to understand and interpret customer inquiries, and respond with accurate and relevant information. Your responses should be clear, concise, and easy to understand, and should reflect the values and goals of the company you represent. Your role is to provide excellent customer service, and to help build and maintain positive relationships with customers.

---

TASK:
Writing product descriptions for new products

N

Most of these responses are slightly more detailed rewrites of the corresponding task. This is a good start, but ultimately not very useful and doesn't demonstrate most of the good practices in prompt design.

## Critique naive responses

In [17]:
constitution = """

1. The prompt should not contain a reference to a customer inquiry, review, or other input from a customer.

""".strip()

# 2. The prompt should be written in English.


def get_llm_critique(constitution, prompt, i):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": f"""Here is a prompt written for a large language model(LLM):
<prompt>
{prompt}
</prompt>

And here are some rules that the prompt is supposed to follow:

<rules>
{constitution}
</rules>

Which rules does this prompt break? If it doesn't break any rules, please respond with "None"."""
            }
        ],
        max_tokens=1024,
        temperature=0.5,
        stream=False
    )
    str = response['choices'][0]['message']['content']
    print(f"Response {i} received")
    return str


with concurrent.futures.ThreadPoolExecutor() as executor:
    critiques = list(executor.map(lambda x: get_llm_critique(
        constitution, x[0], x[1]), [(response, i) for i, response in enumerate(naive_responses)]))

responseCritiqueString = "\n\n---\n\n".join(
    [f"NAIVE PROMPT:\n{response}\n\nCRITIQUE:\n{critique}" for response, critique in zip(naive_responses, critiques)])
print(responseCritiqueString)


Response 2 received
Response 0 received
Response 3 received
Response 1 received
Response 4 received
NAIVE PROMPT:
Please use the following prompt for a large language model:

Task: Reading and responding to customer inquiries via email

Description: As a customer service assistant, your task is to read and respond to customer inquiries via email. Your goal is to provide timely and helpful responses to customers, addressing their concerns and questions in a professional and courteous manner. You will need to be able to understand and interpret customer inquiries, and respond with accurate and relevant information. Your responses should be clear, concise, and easy to understand, and should reflect the values and goals of the company you represent. Your role is to provide excellent customer service, and to help build and maintain positive relationships with customers.

CRITIQUE:
None.

---

NAIVE PROMPT:
Please write product descriptions for our new products. Include all relevant details 